<a href="https://colab.research.google.com/github/BeforeTheStorm202/DACN/blob/main/InSDN_CNN_fgsm_SHAP_detection_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 15.0 MB/s eta 0:00:00


In [3]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Load trainset

In [4]:
df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [5]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

# Load adv samples

In [6]:
fgm_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/24_feature/fgm/fgm_cnn_bin_feature.csv'

In [7]:
fgm_df = pd.read_csv(fgm_path)

In [8]:
fgm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      55151 non-null  float64
 1   Tot Fwd Pkts       55151 non-null  float64
 2   Tot Bwd Pkts       55151 non-null  float64
 3   TotLen Fwd Pkts    55151 non-null  float64
 4   TotLen Bwd Pkts    55151 non-null  float64
 5   Fwd Pkt Len Max    55151 non-null  float64
 6   Fwd Pkt Len Min    55151 non-null  float64
 7   Fwd Pkt Len Mean   55151 non-null  float64
 8   Fwd Pkt Len Std    55151 non-null  float64
 9   Bwd Pkt Len Max    55151 non-null  float64
 10  Bwd Pkt Len Min    55151 non-null  float64
 11  Bwd Pkt Len Mean   55151 non-null  float64
 12  Bwd Pkt Len Std    55151 non-null  float64
 13  Flow Byts/s        55151 non-null  float64
 14  Flow Pkts/s        55151 non-null  float64
 15  Flow IAT Mean      55151 non-null  float64
 16  Flow IAT Std       551

# Load DL-based IDS model

In [9]:
model_cnn = tf.keras.models.load_model('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/insdn_cnn_bin.h5', compile=True)

# SHAP

In [10]:
explainer = shap.KernelExplainer(model_cnn, shap.sample(df, 100, random_state=42))

In [11]:
fgm_100 = fgm_df.head(100)

42m ~ 1g

In [12]:
shap_values = explainer.shap_values(fgm_100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
len(shap_values)

1

In [ ]:
type(shap_values[0])

numpy.ndarray

In [13]:
len(shap_values[0])

100

In [14]:
shap_values[0].shape

(100, 76)

In [15]:
data_df = pd.DataFrame(shap_values[0])

In [16]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.000054,0.000000,0.000000,-0.000064,0.000000,0.000083,0.000000,0.000087,-0.000049,0.000000,...,0.0,0.0,-0.002304,-0.001957,0.000000,0.000389,0.00000,0.000000,0.000000,-0.000751
1,-0.000147,0.000000,0.000000,-0.000076,0.000021,0.000000,0.000036,0.000033,0.000035,0.000000,...,0.0,0.0,0.000000,0.000000,-0.000065,0.000000,0.00000,-0.000792,-0.000477,-0.000353
2,-0.000174,0.000000,0.000000,0.000038,-0.000024,0.000038,-0.000026,-0.000035,-0.000055,0.000000,...,0.0,0.0,0.000031,0.000057,-0.000043,-0.000065,0.00000,-0.000986,-0.000913,0.000098
3,-0.000113,0.000038,0.000000,0.000046,-0.000033,-0.000036,-0.000057,-0.000025,0.000020,-0.000028,...,0.0,0.0,0.000017,-0.000011,0.000000,0.000053,0.00002,-0.001002,-0.000858,0.000076
4,-0.000205,0.000000,0.000079,0.000000,-0.000046,0.000000,0.000000,0.000000,-0.000047,0.000000,...,0.0,0.0,0.000081,0.000000,-0.000072,0.000108,-0.00089,0.000000,0.000000,0.000000


In [17]:
data_df.columns = df.columns
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      100 non-null    float64
 1   Tot Fwd Pkts       100 non-null    float64
 2   Tot Bwd Pkts       100 non-null    float64
 3   TotLen Fwd Pkts    100 non-null    float64
 4   TotLen Bwd Pkts    100 non-null    float64
 5   Fwd Pkt Len Max    100 non-null    float64
 6   Fwd Pkt Len Min    100 non-null    float64
 7   Fwd Pkt Len Mean   100 non-null    float64
 8   Fwd Pkt Len Std    100 non-null    float64
 9   Bwd Pkt Len Max    100 non-null    float64
 10  Bwd Pkt Len Min    100 non-null    float64
 11  Bwd Pkt Len Mean   100 non-null    float64
 12  Bwd Pkt Len Std    100 non-null    float64
 13  Flow Byts/s        100 non-null    float64
 14  Flow Pkts/s        100 non-null    float64
 15  Flow IAT Mean      100 non-null    float64
 16  Flow IAT Std       100 non-

In [18]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/CNN_HSJ/fgm_100_samples_[0]_v2.csv', index=False)

In [19]:
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000054,0.000000,0.000000,-0.000064,0.000000,0.000083,0.000000,0.000087,-0.000049,0.000000,...,0.0,0.0,-0.002304,-0.001957,0.000000,0.000389,0.00000,0.000000,0.000000,-0.000751
1,-0.000147,0.000000,0.000000,-0.000076,0.000021,0.000000,0.000036,0.000033,0.000035,0.000000,...,0.0,0.0,0.000000,0.000000,-0.000065,0.000000,0.00000,-0.000792,-0.000477,-0.000353
2,-0.000174,0.000000,0.000000,0.000038,-0.000024,0.000038,-0.000026,-0.000035,-0.000055,0.000000,...,0.0,0.0,0.000031,0.000057,-0.000043,-0.000065,0.00000,-0.000986,-0.000913,0.000098
3,-0.000113,0.000038,0.000000,0.000046,-0.000033,-0.000036,-0.000057,-0.000025,0.000020,-0.000028,...,0.0,0.0,0.000017,-0.000011,0.000000,0.000053,0.00002,-0.001002,-0.000858,0.000076
4,-0.000205,0.000000,0.000079,0.000000,-0.000046,0.000000,0.000000,0.000000,-0.000047,0.000000,...,0.0,0.0,0.000081,0.000000,-0.000072,0.000108,-0.00089,0.000000,0.000000,0.000000


In [20]:
data_dict = {}
for idx in range(0, 100):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [21]:
print(data_dict)

{'Adv sample 0': ['ACK Flag Cnt', 'Bwd IAT Max', 'SYN Flag Cnt', 'Bwd Pkts/s', 'Fwd Pkts/b Avg', 'Active Min', 'Bwd PSH Flags', 'FIN Flag Cnt', 'Flow Pkts/s', 'Bwd IAT Min'], 'Adv sample 1': ['Bwd Pkts/s', 'Bwd IAT Max', 'Fwd IAT Mean', 'ACK Flag Cnt', 'FIN Flag Cnt', 'Fwd Pkts/b Avg', 'Fwd IAT Tot', 'Fwd IAT Max', 'Bwd IAT Tot', 'Flow Pkts/s'], 'Adv sample 2': ['Bwd Pkts/s', 'Bwd IAT Max', 'FIN Flag Cnt', 'ACK Flag Cnt', 'Fwd IAT Mean', 'Fwd IAT Tot', 'Flow Pkts/s', 'Bwd PSH Flags', 'Fwd IAT Max', 'Flow IAT Max'], 'Adv sample 3': ['Bwd Pkts/s', 'Bwd IAT Max', 'FIN Flag Cnt', 'ACK Flag Cnt', 'Fwd IAT Mean', 'Fwd IAT Tot', 'Flow Pkts/s', 'Fwd IAT Max', 'Bwd PSH Flags', 'Bwd IAT Tot'], 'Adv sample 4': ['Bwd Pkts/s', 'Down/Up Ratio', 'SYN Flag Cnt', 'Fwd IAT Tot', 'Bwd IAT Tot', 'Bwd PSH Flags', 'Flow Pkts/s', 'Flow IAT Mean', 'Active Min', 'Active Mean'], 'Adv sample 5': ['ACK Flag Cnt', 'Bwd IAT Max', 'SYN Flag Cnt', 'Active Max', 'Fwd Pkts/b Avg', 'Bwd IAT Mean', 'Bwd PSH Flags', 'Subf

In [22]:
len(shap_values[0])

100

In [23]:
len(data_dict)

100

# Detection phase

In [24]:
import os
import pandas as pd

In [49]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/cnn_top24_InSDN.csv')
whitelist_df.head(24)

,Feature,Frequency
0,URG Flag Cnt,149
1,Bwd Pkts/s,148
2,FIN Flag Cnt,141
3,Bwd PSH Flags,131
4,SYN Flag Cnt,123
5,PSH Flag Cnt,116
6,ACK Flag Cnt,98
7,Down/Up Ratio,97
8,Flow Pkts/s,86
9,Init Bwd Win Byts,82


## Detection rate (top 24) = 98/100

In [50]:
whitelist = whitelist_df.iloc[:, 0][:24].tolist()
print(whitelist)

['URG Flag Cnt', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Bwd PSH Flags', 'SYN Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Bwd URG Flags', 'Flow Duration', 'Idle Min', 'Bwd Pkt Len Max', 'Fwd IAT Max', 'Flow IAT Std', 'Fwd IAT Mean', 'Flow IAT Max', 'Tot Fwd Pkts', 'Bwd Pkt Len Mean', 'Bwd IAT Std', 'Idle Max']


In [51]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:98


## Detection rate (top 38) = 97/100

In [52]:
whitelist2 = whitelist_df.iloc[:, 0][:38].tolist()
print(whitelist2)

['URG Flag Cnt', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Bwd PSH Flags', 'SYN Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Bwd URG Flags', 'Flow Duration', 'Idle Min', 'Bwd Pkt Len Max', 'Fwd IAT Max', 'Flow IAT Std', 'Fwd IAT Mean', 'Flow IAT Max', 'Tot Fwd Pkts', 'Bwd Pkt Len Mean', 'Bwd IAT Std', 'Idle Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'Idle Mean', 'Bwd Seg Size Avg', 'Pkt Len Mean', 'Flow IAT Mean', 'Pkt Len Std', 'TotLen Bwd Pkts', 'Bwd Pkt Len Std', 'Bwd Header Len', 'Subflow Bwd Pkts', 'Fwd Act Data Pkts', 'Fwd Pkt Len Min', 'Pkt Len Min']


In [53]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:97


# Adv feature vs top normal feature: 7 in common

In [54]:
feature_adv = ['Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
               'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
               'Bwd IAT Min', 'Active Mean', 'Active Std', 'Active Max',
               'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min',
               'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
               'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg']

In [55]:
len(set(whitelist) & set(feature_adv))

7